In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from matplotlib import pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
import seaborn as sns

import branca
import folium
import shared_utils

from siuba import *
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None) 

from calitp import get_engine

engine = get_engine()
connection = engine.connect()

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


# Correlation and VIF in Analytical File

In [2]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/ahsc_grant/'

In [5]:
# start with trips per stop and ridership
analysis_tbl = gpd.read_parquet(f"{GCS_FILE_PATH}analytical_tbl.parquet")
analysis_tbl >> head (5)

,calitp_itp_id,stop_id,geometry,stop_name,n_trips_ampeak,n_trips_midday,n_trips_pmpeak,n_trips_earlyam,n_trips_latepm,n_trips_sat,n_trips_sun,n_trips_weekday,sat_ons,sun_ons,weekday_ons,sum_tracts,sum_total_pop,sum_households,sum_not_us_citizen_pop,sum_black_pop,sum_hispanic_pop,sum_youth_pop,sum_seniors_pop,sum_inc_extremelylow,sum_inc_verylow,sum_inc_low,sum_pop_determined_poverty_status,sum_poverty,sum_no_car,sum_no_cars,sum_land_area,sum_jobs
0,293,670,POINT (19317.277 -398155.080),None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,293,1078,POINT (21987.751 -396450.181),None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,293,217,POINT (27311.975 -399717.150),De La Vina & Canon Perdido,3.0,5.0,4.0,0.0,2.0,12.0,11.0,14.0,104.0,78.0,1552.0,2.0,9570.0,4744.0,1421.0,246.0,3993.0,2674.0,1400.0,1355.0,834.0,905.0,9525.0,1758.0,441.0,872.0,3008305.0,20871.0
3,293,288,POINT (22859.915 -396883.733),La Cumbre & La Colina,11.0,20.0,14.0,0.0,1.0,11.0,11.0,46.0,13.0,8.0,570.0,2.0,9948.0,4340.0,773.0,34.0,1952.0,2613.0,2482.0,594.0,505.0,834.0,9751.0,716.0,130.0,379.0,5282353.0,10611.0
4,293,203,POINT (25145.268 -398574.222),Junipero & Alamar,8.0,10.0,8.0,0.0,1.0,12.0,11.0,27.0,93.0,38.0,911.0,3.0,16624.0,6945.0,1993.0,95.0,5241.0,4185.0,3693.0,920.0,993.0,1245.0,16393.0,1382.0,154.0,529.0,8935899.0,9756.0


In [10]:
# empty stops
empty_stops = (analysis_tbl >> filter(_.stop_name.isnull()))
empty_stops.explore()

In [ ]:
# import matplotlib.pyplot as plt

corr = stops_acs_rollup2.corr()
corr.style.background_gradient(cmap='coolwarm', vmin=-1., vmax=1.)

In [ ]:
import seaborn as sns

sns.lmplot(x="pop_density",y="weekday_ons", data=stops_acs_rollup2).set(title="Weekday Monthly Ridership vs Area Population Density")

In [ ]:
sns.lmplot(x="pct_not_us_citizen_pop",y="weekday_ons", data=stops_acs_rollup2).set(title="Weekday Monthly Ridership vs Non-US-Citizen Population")

In [ ]:
sns.lmplot(x="pct_inc_extremelylow",y="weekday_ons", data=stops_acs_rollup2).set(title="Weekday Monthly Ridership vs Extremely Low Income HH")

In [ ]:
sns.lmplot(x="pct_hh_no_cars",y="weekday_ons", data=stops_acs_rollup2).set(title="Weekday Monthly Ridership vs No-Car HH")